In [1]:
using DataFrames
using CSV
using Plots
using Distributions
using Arrow

In [2]:
include("CommonFunctions.jl")
include("CommonConstants.jl")
using .CommonFunctions
using .CommonConstants

Load up the betas data.

In [3]:
options_folder = option_foldername(currency_type="local", strict_eq=true)

"local-rets_eq-strict"

In [6]:
model_outputs = Dict()
for (currency_risk, benchmark) in COMPLETE_MODELS
    model_filename = joinpath("../data/results", options_folder, "betas/$(benchmark)_$currency_risk.arrow")
    model_outputs[(currency_risk, benchmark)] = DataFrame(Arrow.Table(model_filename))
end

Data loaded in 0.02 seconds.


Extract the info required to calculate p-values and calculate all p-values.

In [8]:
function compute_pvalues(regression_data)
    t = regression_data.coef ./ regression_data.se
    p = ccdf.(Ref(FDist(1,regression_data.df)), t^2)
    
    return p
end

compute_pvalues (generic function with 1 method)

This function computes the proportions of significant p-values by date.

In [14]:
truth_fraction(bool_array) = count(==(true), bool_array)/length(bool_array)

truth_fraction (generic function with 1 method)

In [15]:
function proportion_sig(ptable, sig_level)
    ptable.both_sig = (ptable[:,3] .<= sig_level) .&& (ptable[:,4] .<= sig_level)
    
    date_ptable = groupby(ptable, :date)
    sig_proportions = combine(date_ptable, :both_sig => truth_fraction => :sig_proportion)
    
    return sig_proportions
end

proportion_sig (generic function with 1 method)

This function can be called on a model name to compute the p-values, compute proportions of significant p-values by date, altogether

In [16]:
model = first(COMPLETE_MODELS)

(:lrv_net, :world_capm)

In [17]:
RESULT_COLUMNS

15-element Vector{Symbol}:
 :lrv_net_world_capm_betas
 :verdelhan_world_capm_betas
 :lrv_world_capm_betas
 :lrv_net_world_ff5_betas
 :verdelhan_world_ff5_betas
 :lrv_world_ff5_betas
 :lrv_net_world_ffcarhart_betas
 :verdelhan_world_ffcarhart_betas
 :lrv_world_ffcarhart_betas
 :lrv_net_world_ff6_betas
 :verdelhan_world_ff6_betas
 :lrv_world_ff6_betas
 :lrv_net_world_ff3_betas
 :verdelhan_world_ff3_betas
 :lrv_world_ff3_betas

In [ ]:
x = build_ptable()

In [ ]:
function build